## Imports

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import gym
# from src import agent, dqn, train

## Hyperparams

In [ ]:
# Initialize hyperparams to paper's numbers
N = int(1e7)
epsilon_generator = (1 - (i * .9/1e6) for i in range(int(1e6)))
# mini-batch size = 32
# optimizer RMSProp

## DQN Algo Outline

In [ ]:
replay_mem_size = int(1e6)
mini_batch_size = 32

In [ ]:
agt = agent.DQNAgent()
replay_memory = utils.ReplayMemory(replay_mem_size, mini_batch_size)
obs_history = utils.ObsHistory()

for episode in range(M):  # loop over episodes
    obs_init = env.reset()  # reset environment to start new episode
    obs_history.reset(obs_init)  # reset observations for new episode
    done = False
    
    while not done:  # loop over steps in episode
        phi = obs_history.get_phi()
        a = agt.act(phi)
        obs, rew, done, _ = env.step(a)
        obs_history.store(obs)
        
        # store transition
        phi_1 = obs_history.phi
        replay_memory.store((phi, a, rew, phi_1))
        
        # sample minibatch of transitions
        mini_batch = replay_memory.sample()
        
        # perform a mini-batch of stochastic gradient descent
        grad = 0
        for transition in mini_batch:
            phi_j, a_j, r_j, phi_j1 = transition
            
            # calculate TD error for each transition
            if phi_jp1 is terminal:
                y_j = r_j
            else:
                y_j = r_j + lam * argmax(Q(phi_jp1))
            # calculate the gradient from transition
            grad += get_grad(y_j, Q, phi_j, a_j)
            
        grad = 1/K * grad
        Q.gradient_descent(grad)

### Make DQN agent

In [ ]:
class DQNAgent:
    
    def __init__(self):
        self.s_seq = []
        self.phi_seq = []
        self.replay_mem = ReplayMemory()

        self.q_net = QNet()
        self.epsilon = .9
        
        self.env = gym.envs.make('PongNoFrameskip-v4')
    
    def act(self):
        pass
    
    def reset(self):
        pass
    
    def _train(self):
        pass
    
    def _update_epsilon(self):
        pass

In [ ]:
import numpy as np
from collections import deque

class ReplayMemory:
    
    def __init__(self, N, sample_size):
        self.N = N
        self.sample_size = sample_size
        self.transitions = deque()
        
    def sample(self, k=None):
        if k is None:
            k = self.sample_size
        return np.random.choice(self.transitions, k)

    def add(e):
        if len(self.transitions >= self.N):
            self.transitions.popleft()
        self.transitions.append(e)

### Test random agent on Pong

In [ ]:
import gym
import numpy as np
env = gym.envs.make('PongNoFrameskip-v4')

In [ ]:
s0 = env.reset()

In [ ]:
obss = []
obss.append(env.reset())

for _ in range(1000):
    a = np.random.choice(env.action_space.n)
    obs, rew, done, _ = env.step(a)
    obss.append(obs)
    
    if done:
        obs = env.reset()

In [ ]:
env.action_space.n